In [1]:
import tqdm
import random
import pathlib
import itertools
import collections

import os
import cv2
import numpy as np

import tensorflow as tf
import imageio
from IPython import display
from urllib import request
import matplotlib.pyplot as plt

In [2]:
model= tf.keras.models.load_model("C:/Users/azzaa/Downloads/U101_model.h5", compile=False)


In [3]:
def format_frames(frame, output_size):
    frame = tf.image.convert_image_dtype(frame, tf.float32)
    frame = tf.image.resize_with_pad(frame, *output_size)
    return frame

In [4]:
def frames_from_video_file(video_path, n_frames,  frame_step = 15):
  # Read each video frame by frame
  result = []
  output_size = (224,224)
  src = cv2.VideoCapture(video_path)  

  # ret is a boolean indicating whether read was successful, frame is the image itself
  ret, frame = src.read()
  result.append(format_frames(frame, output_size))

  for _ in range(n_frames - 1):
    for _ in range(frame_step):
      ret, frame = src.read()
    if ret:
      frame = format_frames(frame, output_size)
      result.append(frame)
    else:
      result.append(np.zeros_like(result[0]))
  src.release()
  result = np.array(result)[..., [2, 1, 0]]

  return result

In [5]:
class FrameGenerator:
  def __init__(self, path, n_frames, training = False):
        self.path = path
        self.n_frames = n_frames
  def __call__(self):
    pairs = list(zip(self.path))
    video_frames = frames_from_video_file(self.path, self.n_frames) 
    yield video_frames

In [6]:
data="C:/Users/azzaa/Downloads/videoblocks-extreme-close-up-shot-of-beautiful-young-woman-using-brush-and-doing-eye-makeup_sgggv2drr__adf33cc252de6afa3121c86adabf7939__P360.mp4"

In [7]:
sample_video = frames_from_video_file(data, n_frames = 10)

In [8]:
sample_video=sample_video.reshape([1, 10, 224 ,224, 3])

In [9]:
sample_video.shape

(1, 10, 224, 224, 3)

In [10]:
output_signature = (tf.TensorSpec(shape = ( None, None, None, 3), name='rescaling_input'))
test=tf.data.Dataset.from_generator(FrameGenerator(sample_video, 10, training=True),
                                          output_signature = output_signature)

In [11]:
test

<FlatMapDataset element_spec=TensorSpec(shape=(None, None, None, 3), dtype=tf.float32, name='rescaling_input')>

In [12]:
predicted = model.predict(sample_video)

1/1 [==============================] - 7s 7s/step


In [14]:
predicted

array([[ -0.5390878,  -2.0879474,  -6.533872 ,  -9.150683 ,  -7.50022  ,
        -10.838615 , -13.554056 ,  -9.072392 , -18.556581 , -13.277123 ]],
      dtype=float32)

In [20]:
# predicted = tf.concat(predicted, axis=0)
# predicted = tf.argmax(predicted, axis=1)

In [21]:
np.argmax(predicted[0])

0

In [26]:
names=['ApplyEyeMakeup',
 'ApplyLipstick',
 'Archery',
 'BabyCrawling',
 'BalanceBeam',
 'BandMarching',
 'BaseballPitch',
 'BasketballDunk',
 'Basketball',
 'BenchPress']

In [27]:
res=names[np.argmax(predicted[0])]
res

'ApplyEyeMakeup'